

This is a bit trivial as I already made this demo example a long time ago. I just need to tweak it to match the format of the other datasets.

In [15]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [16]:


df = pd.read_csv('/home/walml/Downloads/example_ring_catalog_basic.csv')  # can be found on dropbox
df

,iauname,ring,smooth-or-featured_featured-or-disk_fraction,disk-edge-on_no_fraction,has-spiral-arms_no_fraction,file_loc
0,J000411.19+020924.0,1,0.634271,0.768214,0.575259,data/example_images/basic/J000411.19+020924.0.jpg
1,J001655.18-002739.1,1,0.706500,0.936972,0.612433,data/example_images/basic/J001655.18-002739.1.jpg
2,J003937.09-032825.0,1,0.693868,0.952298,0.550898,data/example_images/basic/J003937.09-032825.0.jpg
3,J004735.97-005410.2,1,0.613198,0.918429,0.544500,data/example_images/basic/J004735.97-005410.2.jpg
4,J005130.07-011511.2,1,0.650197,0.867980,0.717768,data/example_images/basic/J005130.07-011511.2.jpg
...,...,...,...,...,...,...
465,J144508.61-025056.4,0,0.719779,0.904418,0.629405,data/example_images/basic/J144508.61-025056.4.jpg
466,J165917.60+182032.0,0,0.600055,0.952660,0.756528,data/example_images/basic/J165917.60+182032.0.jpg
467,J145836.50-025637.0,0,0.609062,0.939703,0.763118,data/example_images/basic/J145836.50-025637.0.jpg
468,J095042.39+084353.1,0,0.608941,0.973196,0.571601,data/example_images/basic/J095042.39+084353.1.jpg


In [17]:
df['id_str'] = df['iauname']
del df['iauname']

df['filename'] = df['file_loc'].apply(os.path.basename)
del df['file_loc']

In [18]:
df

,ring,smooth-or-featured_featured-or-disk_fraction,disk-edge-on_no_fraction,has-spiral-arms_no_fraction,id_str,filename
0,1,0.634271,0.768214,0.575259,J000411.19+020924.0,J000411.19+020924.0.jpg
1,1,0.706500,0.936972,0.612433,J001655.18-002739.1,J001655.18-002739.1.jpg
2,1,0.693868,0.952298,0.550898,J003937.09-032825.0,J003937.09-032825.0.jpg
3,1,0.613198,0.918429,0.544500,J004735.97-005410.2,J004735.97-005410.2.jpg
4,1,0.650197,0.867980,0.717768,J005130.07-011511.2,J005130.07-011511.2.jpg
...,...,...,...,...,...,...
465,0,0.719779,0.904418,0.629405,J144508.61-025056.4,J144508.61-025056.4.jpg
466,0,0.600055,0.952660,0.756528,J165917.60+182032.0,J165917.60+182032.0.jpg
467,0,0.609062,0.939703,0.763118,J145836.50-025637.0,J145836.50-025637.0.jpg
468,0,0.608941,0.973196,0.571601,J095042.39+084353.1,J095042.39+084353.1.jpg


In [19]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
train_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/demo_rings/demo_rings_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/demo_rings/demo_rings_test_catalog.parquet'

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

In [23]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    '/home/walml/repos/galaxy-datasets/roots/demo_rings/demo_rings_images.tar.xz'
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

cf650961bcdd889f201ed049fb825085
e84d19d279a02fb3a319d4e8aff4d724
36a75712ce1929b0c376ba3a8d51b7a8
